In [119]:
#importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle as pkl
import dill

In [120]:
#reading the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Pre-processing the data**

In [121]:
#checking for duplicates
df.duplicated().sum()

0

In [122]:
#checking for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [123]:
#dropping irrelevant columns 
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [124]:
# Encoding categorical data
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo = ohe.fit_transform(df["Geography"].values.reshape(-1,1)).toarray()
geo = pd.DataFrame(geo,columns = ohe.get_feature_names_out(["Geography"]))
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [125]:
df.drop(['Geography'],axis=1,inplace=True)

In [126]:
df = pd.concat([df,geo],axis=1)

In [127]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [128]:
#encoding gender
def gender_encode(df):
    return df["Gender"].apply(lambda x: 1 if x =="Male" else 0)

df["Gender"] = gender_encode(df)

In [129]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [130]:
X = df.drop(['Exited'],axis=1)
y = df['Exited']

#splitting the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

#scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


ANN IMPLEMANTATION

In [131]:
#building the model
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [132]:
X_train.shape

(8000, 12)

In [133]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(12,)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')]
)

c:\Users\rashm\Downloads\NLP_KN\ANNProject1\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [134]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [135]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
adam  =Adam(learning_rate = 0.01)
loss = BinaryCrossentropy()

In [136]:
model.compile(optimizer=adam,loss=loss,metrics=["accuracy"])

In [137]:
#setup tensor board
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq = 1)

In [138]:
#setup early stopping
es = EarlyStopping(monitor="val_loss",patience = 10,restore_best_weights=True) 

In [139]:
history = model.fit(X_train,y_train,validation_data =(X_test,y_test),epochs=200,callbacks = [tensorflow_callback,es])

Epoch 1/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8135 - loss: 0.4381 - val_accuracy: 0.8570 - val_loss: 0.3541
Epoch 2/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8550 - loss: 0.3573 - val_accuracy: 0.8630 - val_loss: 0.3408
Epoch 3/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8579 - loss: 0.3406 - val_accuracy: 0.8680 - val_loss: 0.3376
Epoch 4/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8535 - loss: 0.3510 - val_accuracy: 0.8610 - val_loss: 0.3387
Epoch 5/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8594 - loss: 0.3472 - val_accuracy: 0.8580 - val_loss: 0.3358
Epoch 6/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8680 - loss: 0.3277 - val_accuracy: 0.8610 - val_loss: 0.3374
Epoch 7/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.3298 - val_accuracy: 0.8495 - val_loss: 0.3566
Epoch 8/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8660 - loss: 0.3199 - val_accu

In [140]:
model.save('ann.h5')

In [141]:
#load tensor board
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [142]:
%tensorboard --logdir logs/fit20240926-160122

Reusing TensorBoard on port 6007 (pid 6116), started 3:18:41 ago. (Use '!kill 6116' to kill it.)

In [143]:
!kill 6116

kill: 6116: No such process


**EDA**

In [144]:
import matplotlib.pyplot as plt

In [145]:
with open('ohe.pkl','wb') as f:
    pkl.dump(ohe,f)
with open('gender_encode.dill','wb') as f:
    dill.dump(gender_encode,f)
with open('sc.pkl','wb') as f:
    pkl.dump(sc,f)